# [Data Leakage](https://www.kaggle.com/dansbecker/data-leakage)

## What is it?

Data leakage is one of the most important issues for a data scientist to understand.  
If you don't know how to prevent it, leakage will come up frequently, and it will ruin your models in the most subtle and dangerous ways.  
Specifically, leakage causes a model to look accurate until you start making decisions with the model, and then the model becomes very inaccurate.  
This tutorial will show you what leakage is and how to avoid it.  
There are two main types of leakage: **Leaky Predictors** and **Leaky Validation Strategies**.

### Leaky Predictors

This occurs when your predictors include data that will not be available at the time you make your predictions.  
For example, imagine that you want to predict who will catch pneumonia.  
The first few rows of your raw data might look like this:

![data_leakage1](img/data_leakage1.png)

People take antibiotic medicines after getting pneumonia in order to recover.  
So the raw data shows a strong relationship between those columns.  
But `took_antibiotic_medicine` is frequently changed after the value for `got_pneumonia` is determined.  
This is target leakage.  
The model would see that anyone who has a value of `False` for `took_antibiotic_medicine` didn't have pneumonia.  
Validation data comes from the same source, so the pattern will repeat itself in validation, and the model will have great validation (or cross-validation) scores.  
However, the model will be less accurate when subsequently deployed in the real world.  
To prevent this type of data leakage, any variable updated (or created) after the target value is realized should be excluded.  
Because when we use this model to make new predictions, that data won't be available to the model.

![data_leakage2](img/data_leakage2.png)

### Leaky Validation Strategies